In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
=========================================================
Importance of Feature Scaling
=========================================================

In [ ]:
Feature scaling through standardization (or Z-score normalization)
can be an important preprocessing step for many machine learning
algorithms. Standardization involves rescaling the features such
that they have the properties of a standard normal distribution
with a mean of zero and a standard deviation of one.

In [ ]:
While many algorithms (such as SVM, K-nearest neighbors, and logistic
regression) require features to be normalized, intuitively we can
think of Principle Component Analysis (PCA) as being a prime example
of when normalization is important. In PCA we are interested in the
components that maximize the variance. If one component (e.g. human
height) varies less than another (e.g. weight) because of their
respective scales (meters vs. kilos), PCA might determine that the
direction of maximal variance more closely corresponds with the
'weight' axis, if those features are not scaled. As a change in
height of one meter can be considered much more important than the
change in weight of one kilogram, this is clearly incorrect.

In [ ]:
To illustrate this, PCA is performed comparing the use of data with
:class:`StandardScaler <sklearn.preprocessing.StandardScaler>` applied,
to unscaled data. The results are visualized and a clear difference noted.
The 1st principal component in the unscaled set can be seen. It can be seen
that feature #13 dominates the direction, being a whole two orders of
magnitude above the other features. This is contrasted when observing
the principal component for the scaled version of the data. In the scaled
version, the orders of magnitude are roughly the same across all the features.

In [ ]:
The dataset used is the Wine Dataset available at UCI. This dataset
has continuous features that are heterogeneous in scale due to differing
properties that they measure (i.e alcohol content, and malic acid).

In [ ]:
The transformed data is then used to train a naive Bayes classifier, and a
clear difference in prediction accuracies is observed wherein the dataset
which is scaled before PCA vastly outperforms the unscaled version.


<br>
from sklearn.model_selection import train_test_split<br>
from sklearn.preprocessing import StandardScaler<br>
from sklearn.decomposition import PCA<br>
from sklearn.naive_bayes import GaussianNB<br>
from sklearn import metrics<br>
import matplotlib.pyplot as plt<br>
from sklearn.datasets import load_wine<br>
from sklearn.pipeline import make_pipeline<br>
print(__doc__)<br>
# Code source: Tyler Lanigan <tylerlanigan@gmail.com><br>
#              Sebastian Raschka <mail@sebastianraschka.com><br>
# License: BSD 3 clause<br>
RANDOM_STATE = 42<br>
FIG_SIZE = (10, 7)<br>
features, target = load_wine(return_X_y=True)<br>
# Make a train/test split using 30% test size<br>
X_train, X_test, y_train, y_test = train_test_split(features, target,<br>
                                                    test_size=0.30,<br>
                                                    random_state=RANDOM_STATE)<br>
# Fit to data and predict using pipelined GNB and PCA.<br>
unscaled_clf = make_pipeline(PCA(n_components=2), GaussianNB())<br>
unscaled_clf.fit(X_train, y_train)<br>
pred_test = unscaled_clf.predict(X_test)<br>
# Fit to data and predict using pipelined scaling, GNB and PCA.<br>
std_clf = make_pipeline(StandardScaler(), PCA(n_components=2), GaussianNB())<br>
std_clf.fit(X_train, y_train)<br>
pred_test_std = std_clf.predict(X_test)<br>
# Show prediction accuracies in scaled and unscaled data.<br>
print('\nPrediction accuracy for the normal test dataset with PCA')<br>
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test)))<br>
print('\nPrediction accuracy for the standardized test dataset with PCA')<br>
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test_std)))<br>
# Extract PCA from pipeline<br>
pca = unscaled_clf.named_steps['pca']<br>
pca_std = std_clf.named_steps['pca']<br>
# Show first principal components<br>
print('\nPC 1 without scaling:\n', pca.components_[0])<br>
print('\nPC 1 with scaling:\n', pca_std.components_[0])<br>
# Use PCA without and with scale on X_train data for visualization.<br>
X_train_transformed = pca.transform(X_train)<br>
scaler = std_clf.named_steps['standardscaler']<br>
X_train_std_transformed = pca_std.transform(scaler.transform(X_train))<br>
# visualize standardized vs. untouched dataset with PCA performed<br>
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=FIG_SIZE)<br>
for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):<br>
    ax1.scatter(X_train_transformed[y_train == l, 0],<br>
                X_train_transformed[y_train == l, 1],<br>
                color=c,<br>
                label='class %s' % l,<br>
                alpha=0.5,<br>
                marker=m<br>
                )<br>
for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):<br>
    ax2.scatter(X_train_std_transformed[y_train == l, 0],<br>
                X_train_std_transformed[y_train == l, 1],<br>
                color=c,<br>
                label='class %s' % l,<br>
                alpha=0.5,<br>
                marker=m<br>
                )<br>
ax1.set_title('Training dataset after PCA')<br>
ax2.set_title('Standardized training dataset after PCA')<br>
for ax in (ax1, ax2):<br>
    ax.set_xlabel('1st principal component')<br>
    ax.set_ylabel('2nd principal component')<br>
    ax.legend(loc='upper right')<br>
    ax.grid()<br>
plt.tight_layout()<br>
plt.show()